### based on https://til.simonwillison.net/llms/python-react-pattern

In [8]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [9]:
client = OpenAI()

In [5]:
chat_completion = client.chat.completions.create(
    model="gpt-4o-mini", messages=[{"role": "user", "content": "Hello World"}]
)

In [6]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

### Now, let's build a tool-calling agent

In [1]:
class Agent:
    def __init__(self, system=""):
        # Initialize the system prompt (optional)
        self.system = system
        # Initialize the conversation history as a list of messages
        self.messages = []
        # If a system prompt is provided, add it to the message history
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        # Add the user's message to the conversation history
        self.messages.append({"role": "user", "content": message})
        # Generate a response from the assistant
        result = self.execute()
        # Add the assistant's response to the conversation history
        self.messages.append({"role": "assistant", "content": result})
        # Return the response to the caller
        return result

    def execute(self):
        # Call the OpenAI chat completion API using the current message history
        completion = client.chat.completions.create(
            model="gpt-4o-mini",  # Specify the model to use
            temperature=0,  # Use deterministic output (no randomness)
            messages=self.messages,  # Provide full conversation context
        )
        # Return the assistant's message content from the first choice
        return completion.choices[0].message.content

### Now we will create a ReAct agent, a ReAct agent will take very specific system message (Prompt).

It will use `thought` to describe its thoughts about the question it been asked, it will then use `action` to run one of the actions available to it, it will then return `pause`.

After that, `observation` will be used to signal the result of running those actions. We then tell it what the available actions are. In this example, we give it access to calculate an average dog weight.

In [2]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [3]:
def calculate(what):
    """
    Evaluate a string expression using Python's built-in eval() function.

    Parameters:
        what (str): A string containing a Python expression to be evaluated.

    Returns:
        The result of evaluating the expression.

    WARNING:
        This function uses eval(), which can execute arbitrary code.
        Do NOT use this with untrusted input, as it can lead to severe
        security vulnerabilities such as code injection.
    """
    return eval(what)


def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weights 50 lbs"


known_actions = {"calculate": calculate, "average_dog_weight": average_dog_weight}

In [5]:
my_bot = Agent(prompt)

In [ ]:
result = my_bot("How much does a toy poodle weigh?")
print(
    result
)  # this will output Action to let you know that you (as user) should use average_dog_weight function to lookup how much a Toy Poodle wights

Thought: I need to find the average weight of a Toy Poodle using the average_dog_weight action. 
Action: average_dog_weight: Toy Poodle
PAUSE


In [13]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [14]:
next_prompt = "Observation: {}".format(result)

In [15]:
my_bot(next_prompt)
my_bot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

Next example with more complex question

In [ ]:
my_bot = Agent(prompt)  # to clean prev histories

question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""

my_bot(question)

"Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll use the average_dog_weight action for both breeds. \n\nAction: average_dog_weight: Border Collie\nPAUSE"

The answer from above code is 

"Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll use the average_dog_weight action for both breeds. \n\nAction: average_dog_weight: Border Collie\nPAUSE"

In [17]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [18]:
my_bot(next_prompt)

'Thought: Now I need to find the average weight of a Scottish Terrier to complete the calculation for their combined weight.\n\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [19]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [20]:
my_bot(next_prompt)

'Thought: I now have the average weights for both dogs: a Border Collie weighs 37 lbs and a Scottish Terrier weighs 20 lbs. I can calculate their combined weight by adding these two values together.\n\nAction: calculate: 37 + 20\nPAUSE'

In [21]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [22]:
my_bot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### Now let's automate this tool-calling agent

In [28]:
action_re = re.compile(r"^Action: (\w+): (.*)$")

```
^             → Matches the start of the string  
Action:       → Matches the literal text "Action:"  
(\w+)         → Captures one or more word characters (letters, digits, or underscores); this represents the action name  
:             → Matches a literal colon  
(.*)          → Captures any remaining characters until the end of the line; this represents the action input or arguments  
$             → Matches the end of the string  
```

Example

```py
line = "Action: Search: what is the capital of France?"
match = action_re.match(line)

if match:
    action_name = match.group(1)  # "Search"
    action_input = match.group(2)  # "what is the capital of France?"
```

In [ ]:
def query(question, max_turns=5):
    """
    Conduct a multi-turn interaction with an agent, optionally executing actions
    returned in the agent's response.

    Parameters:
        question (str): The initial user question to ask the agent.
        max_turns (int): Maximum number of turns (loop iterations) allowed.

    Returns:
        None or final response if no action is found.
    """
    i = 0  # turn counter
    bot = Agent(prompt)  # instantiate the agent with an optional system prompt
    next_prompt = question  # start with the user's question

    while i < max_turns:
        i += 1
        result = bot(next_prompt)  # send the prompt to the agent and get its response
        print(result)

        # Try to extract action lines using regex, e.g., "Action: Search: query"
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]

        # "Action: average_dog_weight: Border Colli" -> average_dog_weight, Border Colli
        # "Action: average_dog_weight: Scottish Terrier" -> average_dog_weight, Scottish Terrier
        # "Action: calculate: 37 + 20" -> calculate, 37 + 20

        if actions:
            # Action found in response
            action, action_input = actions[0].groups()

            # Ensure the action is known and supported
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))

            print(" -- running {} {}".format(action, action_input))

            # Execute the corresponding action with the provided input
            observation = known_actions[action](action_input)
            print("Observation:", observation)

            # Feed the observation back into the agent for the next turn
            next_prompt = "Observation: {}".format(observation)
        else:
            # No action detected in the response — terminate
            return

In [27]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""

query(question)

Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll use the average_dog_weight action for both breeds. 

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier to complete the calculation for their combined weight.

Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights for both dogs: a Border Collie weighs 37 lbs and a Scottish Terrier weighs 20 lbs. I can calculate their combined weight by adding these two values together.

Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
